# Projeto — ComfyUI / Stable Diffusion (documentado)

## Objetivo do projeto
Este notebook implementa um *pipeline* que usa **Stable Diffusion** via ComfyUI (ou downloads de checkpoints) para gerar imagens. O objetivo desta documentação é atender aos requisitos do trabalho: **descrever a arquitetura dos modelos, explicar os blocos usados no fluxo, e fornecer instruções para reproducibilidade e execução**.

> Critérios importantes (resumido):
> - Explicar arquitetura e blocos do pipeline (entrada → codificadores CLIP → difusão → VAE → saída).
> - Incluir exemplos de entrada/saída e parâmetros usados.
> - Mencionar dependências e instruções para executar o notebook/ComfyUI localmente.
> - Opcional: uso de LoRA, ControlNet, etc. (adicionar se aplicável).


## Sumário rápido do que há neste notebook
- Preparação do ambiente (variáveis, paths).
- Download automático de checkpoints/VAE/CLIP.
- Inicialização / controle do ComfyUI (scripts para checar logs / iniciar servidor).

## Como usar este notebook (instruções rápidas)
1. Coloque este notebook na mesma pasta onde está a instalação do **ComfyUI** ou ajuste a variável `BASE` nas células para apontar para o diretório correto.  
2. Garanta que o ambiente virtual com PyTorch + CUDA está ativado (ou rode o ComfyUI com `--lowvram --force-fp16` se necessário).  
3. Execute as células na ordem, lendo os comentários e observando mensagens de log.  
4. Se houver downloads grandes (checkpoints ~5–10 GB), verifique espaço em disco e integridade do arquivo (.safetensors completo).


## Arquitetura (descrição genérica aplicada ao notebook)
Um pipeline de Text→Image típico contém os seguintes blocos (e mapeamento para o que o notebook / ComfyUI faz):

- **Prompt / Texto de entrada** → alimenta o *text encoder* (CLIP / text encoder do modelo).
- **Text encoder (CLIP)** → produz embeddings condicionais para o modelo de difusão.
- **U-Net / Diffusion Model** → gera latentes condicionados pelos embeddings.
- **VAE (Decoder)** → converte latentes para imagens RGB finais.
- **(Opcional) LoRA / Hypernetwork** → adapta pesos finos para estilos ou personagens específicos.
- **(Opcional) ControlNet / Conditioning** → adiciona condicionamento estrutural (esboço, canny, pose).
- **Output** → upscale / save / export (imagem final).

<!-- Célula original markdown #0 preservada -->

# *COMFYUI MANAGER + NODES* 

### Explicação da célula de código
**Objetivo:** Baixar/checkpoints/Modelos (checkpoint, VAE, CLIP). Esta célula realiza o download de arquivos de modelos e salva em `models/checkpoints/`. Verifique espaço em disco e integridade do arquivo após o download.

**Observações para execução:**
- Execute em ordem.
- Se a célula faz download, confirme o arquivo final em `models/checkpoints/`.
- Se houver falhas de VRAM, considere reiniciar ComfyUI com `--lowvram --force-fp16`.

In [ ]:
import subprocess
import os
from os import path
import warnings
from IPython.display import clear_output
warnings.filterwarnings('ignore')
from multiprocessing import Process
import sys
import time

BASE = os.getcwd()

def download_from_civitai(URL_CITVITAI, PASTA_ONDE_DEVE_SER_INSTALADA, NOME_DO_ARQUIVO):
    tok = COLE_SEU_TOKEB_CTIVITAI_AQUI
    model_url = f"{url}&token={tok}"
    save_path = f"{BASE}/ComfyUI/models/{folder}/{name}"
    get_ipython().system(f'wget -O "{save_path}" "{model_url}"')


def delete_model(PASTA, NOME):
    %cd $BASE/ComfyUI/models/$folder
    all_names = os.listdir()
    print("O caminho atual é: ", os.getcwd())
    print("Todos os arquivos no diretório atual: ", all_names)
    
    if name in all_names:
        %rm $name
        print("O ARQUIVO DO MODELO FOI EXCLUÍDO COM SUCESSO")
    else:
        print("O ARQUIVO DO MODELO FORNECIDO NÃO FOI ENCONTRADO")

%cd $BASE
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
get_ipython().system('git pull')
!pip install - torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu124
!pip install -r requirements.txt
!pip install --upgrade gguf
!pip install bitsandbytes>=0.43.0
!mamba install openssh -y
 
%cd $BASE/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager
get_ipython().system('git pull')
!pip install -r requirements.txt

%cd $BASE/ComfyUI/custom_nodes
!git clone https://github.com/chrisgoringe/cg-use-everywhere.git  
!git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git  
!git clone https://github.com/WASasquatch/was-node-suite-comfyui.git  
!git clone https://github.com/rgthree/rgthree-comfy.git  
!git clone https://github.com/city96/ComfyUI-GGUF  
!git clone https://github.com/crystian/ComfyUI-Crystools.git  
%cd ComfyUI-Crystools
!pip install -r requirements.txt
!git clone https://github.com/kijai/ComfyUI-KJNodes.git
!git clone https://github.com/11cafe/comfyui-workspace-manager.git
%cd /kaggle/working/ComfyUI/custom_nodes
!pip install -r requirements.txt


clear_output(wait=True)
%cd $BASE/ComfyUI

print(f"Current path: {BASE}/ComfyUI")
print("COMFYUI MANAGER + CUSTOM NODES FORAM INSTALADOS COM SUCESSO!")

<!-- Célula original markdown #2 preservada -->

# MODELS + CLIP + VAE

<!-- Documentação automática para a célula de código #3 -->

### Explicação da célula de código
**Objetivo:** Baixar/checkpoints/Modelos (checkpoint, VAE, CLIP). Esta célula realiza o download de arquivos de modelos e salva em `models/checkpoints/`. Verifique espaço em disco e integridade do arquivo após o download.

**Observações para execução:**
- Execute em ordem.
- Se a célula faz download, confirme o arquivo final em `models/checkpoints/`.
- Se houver falhas de VRAM, considere reiniciar ComfyUI com `--lowvram --force-fp16`.

---


In [ ]:
### Dreamshaper_8
model_url = "https://civitai.com/api/download/models/128713?type=Model&format=SafeTensor&size=pruned&fp=fp16"
model_name = "dreamshaper_8.safetensors"
%cd /kaggle/working/ComfyUI/models/checkpoints
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

### blindbox_V1Mix
model_url = "https://civitai.com/api/download/models/32988?type=Model&format=SafeTensor&size=full&fp=fp16"
model_name = "blindbox_V1Mix.safetensors"
%cd /kaggle/working/ComfyUI/models/loras
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

%cd /kaggle/working/ComfyUI
print("FLUX + CLIP + VAE FORAM INSTALADOS COM SUCESSO! 👍")

<!-- Célula original markdown #4 preservada -->

# START COMFYUI

<!-- Documentação automática para a célula de código #5 -->

### Explicação da célula de código
**Objetivo:** Iniciar ou controlar o ComfyUI localmente. Pode criar/abrir logs, iniciar o servidor e monitorar a saída.

**Observações para execução:**
- Execute em ordem.
- Se a célula faz download, confirme o arquivo final em `models/checkpoints/`.
- Se houver falhas de VRAM, considere reiniciar ComfyUI com `--lowvram --force-fp16`.

---


In [ ]:
%cd $BASE/ComfyUI

!touch log.txt
open('log.txt', 'w').close()
clear_output(wait=True)

def run_app():
    cmd = f"python {BASE}/ComfyUI/main.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8188 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

<!-- Documentação automática para a célula de código #6 -->

### Explicação da célula de código
**Objetivo:** Célula de execução do pipeline. Revise o código para entender operações específicas.

**Observações para execução:**
- Execute em ordem.
- Se a célula faz download, confirme o arquivo final em `models/checkpoints/`.
- Se houver falhas de VRAM, considere reiniciar ComfyUI com `--lowvram --force-fp16`.

---
